In [36]:
import numpy as np
import random

# 5.4. 간단한 역전파 구현

In [15]:
# 곱셈 계층 구현
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None
    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x*y
        return out
    def backward(self, dout):
        dx = dout * self.y
        dy = dout * self.x
        return dx, dy

apple = 100
apple_num=2
tax=1.1
mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()
#순전파
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)
print("순전파: ", price)
#역전파
dprice=1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)
print("역전파: ", dapple, dapple_num, dtax)

순전파:  220.00000000000003
역전파:  2.2 110.00000000000001 200


In [18]:
#덧셈 계층 구현
class AddLayer:
    def __init__(self):
        pass
    def forward(self, x, y):
        out = x+y
        return out
    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1
        return dx, dy
apple = 100
apple_num=2
orange = 150
orange_num = 3
tax = 1.1
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add = AddLayer()
mul_tax_layer = MulLayer()
#순전파
apple_price = mul_apple_layer.forward(apple, apple_num)
orange_price = mul_orange_layer.forward(orange, orange_num)
all_price = add.forward(apple_price, orange_price)
price = mul_tax_layer.forward(all_price, tax)
#역전파
dprice=1
dall_price, dtax = mul_tax_layer.backward(dprice)
dapple_price, dorange_price = add.backward(dall_price)
dorange, dorange_num = mul_orange_layer.backward(dorange_price)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)
print(price)
print("역전파: ", dapple_num, dapple, dorange, dorange_num, dtax)

715.0000000000001
역전파:  110.00000000000001 2.2 3.3000000000000003 165.0 650


# 5.5. 활성화 함수 계층 구현

In [21]:
# ReLU 계층
class Relu:
    def __init__(self):
        self.mask = None
    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0
        return out
    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout
        return dx
#sigmoid 계층
class Sigmoid:
    def __init__(self):
        self.out = None
    def forward(self, x):
        out = 1/(1+np.exp(-x))
        self.out = out
    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out
        return dx

# 5.6. Affine/Softmax 계층 구현

In [25]:
#배치용 Affine 계층 
x_dot_w = np.array([ [0, 0, 0], [10, 10, 10]])
b = np.array([1, 2, 3])
print(x_dot_w+b)

dy = np.array([[1, 2, 3], [4, 5, 6]])
db = np.sum(dy, axis=0)
print(db)

class Affine:
    def __init__(self, w, b):
        self.w = w
        self.b = b
        self.x = None
        self.dw = None
        self.db = None
    def forward(self, x):
        self.x = x
        out = np.dot(x, self.w) + self.b
        return out
    def backward(self, dout):
        dx = np.dot(dout, self.w.T)
        self.dw = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)
        return dx


[[ 1  2  3]
 [11 12 13]]
[5 7 9]


In [31]:
#Softmax-with-Loss 계층
def cross_entropy_error(y, t):
    delta = 1e-7
    return -np.sum(t*np.log(y+delta))
class SoftmaxWithLoss:
    def __init__(self):
        self.loss=None
        self.y = None
        self.t = None
    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        return self.loss
    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size
        return dx
    def gradient(self, x, t):
        self.loss(x, t)
        dout = 1
        dout = self.lastLayer.backward(dout)
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)
        grads = {}
        grads['w1'] = self.layers['Affine1'].dw
        grads['b1'] = self.layers['Affine2'].db
        grads['w2'] = self.layers['Affine3'].dw
        grads['b2'] = self.layers['Affine4'].db
        return grads

In [ ]:
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)
network = TwoLayerNet(784, 50, 10)
iters_num = 1000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1
train_loss_list = []
train_acc_list = []
test_acc_list = []
iter_per_epoch = max(train_size / batch_size, 1)
iter_per_epoch = 10
for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_trai n[batch_mask]
    t_batch = t_train[batch_mask]
    grad = network.gradient(x_batch, t_batch)
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print("train acc, test acc | " + str(train_acc) + ", " + str(test_acc))        

# 오늘의 과제??

In [54]:
#배치용 Affine 계층 
class Affine:
    def __init__(self, w, b):
        self.w = w
        self.b = b
        self.x = None
        self.dw = None
        self.db = None
        
    def forward(self, x):
        self.x = x
        out = np.dot(x, self.w) + self.b
        return out
    def backward(self, dout):
        dx = np.dot(dout, self.w.T)
        self.dw = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)
        return dx

input_d = np.random.randint(0, 10, (6, 5))
weight = np.random.uniform(-3, 3, (5, 3))
weight = np.round(weight, decimals=2)
bias = 1

af = Affine(weight, b)
af_forward = af.forward(input_d)
af_backward = af.backward(af_forward)

print("input_d = ", input_d.shape)
print("\naf_forward: \n", af_forward, af_forward.shape)
print("\naf_backward: \n", af_backward, af_backward.shape)


input_d =  (6, 5)

af_forward: 
 [[-19.36  19.51  10.44]
 [ 11.37  15.87   9.01]
 [ -1.62   1.94  -1.38]
 [  9.46  23.29  16.8 ]
 [ -4.12  12.33  10.74]
 [  1.56  -1.2   -1.42]] (6, 3)

af_backward: 
 [[-71.2901  48.7429 120.2611 -60.8635  -7.5436]
 [-42.2397 -30.5395  46.7237  -5.0663  68.692 ]
 [ -3.203    7.6446   5.5902  -9.2122  -1.096 ]
 [-71.8701 -31.6141  91.0655 -13.6671  82.6902]
 [-46.4413   3.8995  70.2403 -19.1447  19.579 ]
 [  5.8002  -2.792   -9.9686   3.1634   1.0982]] (6, 5)
